In [1]:
import numpy as np
import pandas as pd
import argparse
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

from custom_dataset import CustomDataset
from network import Network
from utils import *

from sklearn.model_selection import StratifiedShuffleSplit

/data/users2/pnadigapusuresh1/software/envs/latest/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
torch.cuda.device_count()

2

In [3]:
parent_directory = '/data/users2/pnadigapusuresh1/JobOutputs'
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


### Loading model for 1 Fully-connected layer. 

In [4]:

model = Network()
#model = nn.DataParallel(model)
model.to(device)

# Loading the model from Job 5436878
#loading model from 6066159
load_path = os.path.join(parent_directory,'5436878','models','epoch_28')

model.load_state_dict(torch.load(load_path))

<All keys matched successfully>

In [5]:
model = nn.DataParallel(model)

In [4]:
torch.manual_seed(52)
np.random.seed(52)
# number of subprocesses to use for data loading
num_workers = 1
# how many samples per batch to load
batch_size = 5

In [7]:
valid_size = 0.20
# percentage of data to be used for testset
test_size = 0.10


train_data = CustomDataset(transform = 
                        transforms.Compose([
                            transforms.RandomHorizontalFlip()
                            ]),train=True)

valid_data = CustomDataset(train=False)

# get filtered variables
vars = valid_data.vars

In [9]:
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
test_split = int(np.floor(test_size * num_train))
test_idx, train_idx = indices[: test_split], indices[test_split : ]

train_rem = len(train_idx)
valid_spilt = int(np.floor(valid_size * train_rem))

valid_idx, train_idx = indices[: valid_spilt], indices[valid_spilt : ]

In [10]:
test_sampler = SubsetRandomSampler(test_idx)
test_loader = DataLoader(valid_data,batch_size = batch_size, 
                            sampler = test_sampler, num_workers=num_workers)

In [11]:
criterion = nn.L1Loss()

In [16]:
def test(model):
    actual_test = torch.tensor([]).to(device)
    pred_test = torch.tensor([]).to(device)

    test_loss = 0

    for X, y in test_loader:
        X,y = X.to(device),y.to(device)
        actual_test = torch.cat((actual_test,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        try:
            test_loss += criterion(pred,y.float()).item()
        except:
            print(pred)
            print(y)
    
    return test_loss/len(test_loader)

### 1 Fc Layer/Folds

### Testing for 1 FC layers

In [12]:
train_scores = [3.014,2.927,2.990,2.935,2.883]
val_scores = [2.960,3.033,3.071,2.920,3.011]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 2.9498000000000006
train std = 0.04677349676900367
val mean = 2.999
val std = 0.0533778980477876


In [13]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,1))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=1, bias=Tr

In [14]:
best_models_1fc = {
    '1' : 'epoch_65',
    '2' : 'epoch_71',
    '3' : 'epoch_65',
    '4' : 'epoch_71',
    '5' : 'epoch_75'
}

In [17]:
test_scores = []
for k,v in best_models_1fc.items():
    load_path = os.path.join(parent_directory,'6164265/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    test_scores.append(test(model))

print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

[3.579393994208402, 3.0456600812246215, 3.382453809894106, 3.4724090444966684, 2.9856867243658822]
mean =  3.2931207308379356
std =  0.23572421224453227


### Testing for 2 Fc layers

In [15]:
train_scores = [4.956, 4.413, 4.980, 4.451, 4.669]
val_scores = [4.854, 4.495, 4.834, 4.397, 4.498]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 4.6938
train std = 0.24044741628888447
val mean = 4.6156
val std = 0.1901016570153978


In [16]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),
    nn.Dropout(),nn.Linear(256,1))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [18]:
best_models_2fc = {
    '1' : 'epoch_26',
    '2' : 'epoch_74',
    '3' : 'epoch_25',
    '4' : 'epoch_66',
    '5' : 'epoch_48'
}

In [19]:
test_scores = []
for k,v in best_models_2fc.items():
    load_path = os.path.join(parent_directory,'6173079/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    test_scores.append(test(model))


print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

[5.129466643123506, 6.777093748176623, 6.380527339116583, 5.641125306843212, 4.606362208555329]
mean =  5.70691504916305
std =  0.7934768392083877


### Testing for 3 Fc layers

In [32]:
train_scores = [7.314, 6.267, 6.291, 6.107, 6.374]
val_scores = [8.838, 6.731, 6.394, 5.996, 6.227]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 6.4706
train std = 0.4304809403446335
val mean = 6.837200000000001
val std = 1.0286755367947655


In [20]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),
    nn.Dropout(),nn.Linear(256,128),nn.ReLU(),nn.Dropout(),
    nn.Linear(128,1))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [21]:
best_models_3fc = {
    '3' : 'epoch_62',
    '4' : 'epoch_75',
    '5' : 'epoch_51'
}

In [22]:
test_scores = []
for k,v in best_models_3fc.items():
    load_path = os.path.join(parent_directory,'6183834/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    test_scores.append(test(model))

print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores))

[8.52802372578555, 7.74477335012184, 7.73857762078819]
mean =  8.003791565565194
std =  0.37069674499287614


# Testing for 4 FC layers

In [24]:
train_scores = [9.393, 8.197, 8.541, 8.167, 8.187]
val_scores = [9.414, 8.256, 8.261, 8.059, 8.179]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 8.497
train std = 0.4689886992241929
val mean = 8.433800000000002
val std = 0.49551242164046705


In [23]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),
    nn.Dropout(),nn.Linear(256,128),nn.ReLU(),nn.Dropout(),
    nn.Linear(128,64),nn.ReLU(),nn.Dropout(),nn.Linear(64,1))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [27]:
best_models_4fc = {
    '1' : 'epoch_19',
    '2' : 'epoch_68',
    #'3' : 'epoch_58',
    '4' : 'epoch_67',
    '5' : 'epoch_75'
}

In [28]:
test_scores = []
for k,v in best_models_4fc.items():
    load_path = os.path.join(parent_directory,'6189282/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    test_scores.append(test(model))

print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores))

[11.75920408176926, 10.971568537958014, 15.077527684985467, 12.054282779813562]
mean =  12.465645771131577
std =  1.559044040566191


# Testing for 1 FC - Working memory classification

In [4]:
valid_size = 0.20
# percentage of data to be used for testset
test_size = 0.10


train_data = CustomDataset(transform = 
                        transforms.Compose([
                            transforms.RandomHorizontalFlip()
                            ]),train=True)

valid_data = CustomDataset(train=False,test=True)

# get filtered variables
vars = valid_data.vars

In [5]:
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
test_split = int(np.floor(test_size * num_train))
test_idx, train_idx = indices[: test_split], indices[test_split : ]

train_rem = len(train_idx)
valid_spilt = int(np.floor(valid_size * train_rem))

valid_idx, train_idx = indices[: valid_spilt], indices[valid_spilt : ]

In [6]:
len(test_idx)

421

In [8]:
test_sampler = SubsetRandomSampler(test_idx)
test_loader = DataLoader(valid_data,batch_size = 13, 
                            sampler = test_sampler, num_workers=2)

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
def test(model):
    actual_test = torch.tensor([]).to(device)
    pred_test = torch.tensor([]).to(device)

    test_loss = 0
    num_correct_test = 0

    for X, y in test_loader:
        X,y = X.to(device),y.to(device)
        actual_test = torch.cat((actual_test,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        pred_test = torch.cat((pred_test,torch.max(F.softmax(pred,dim=1), dim=1)[1]),0)
        try:
            test_loss += criterion(pred,y).item()
            correct = torch.eq(torch.max(F.softmax(pred,dim=1), dim=1)[1],y).view(-1)
            num_correct_test += torch.sum(correct).item()

        except:
            print(pred)
            print(y)
    
    return test_loss/len(test_loader), num_correct_test/len(test_idx)

In [11]:
train_scores = [0.775, 0.693, 0.715, 0.699, 0.718]
val_scores = [0.661,0.650, 0.669, 0.673, 0.662]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 0.72
train std = 0.029065443399335946
val mean = 0.663
val std = 0.007874007874011818


In [12]:
best_models_1fc = {
    '1' : 'epoch_29',
    '2' : 'epoch_10',
    '3' : 'epoch_12',
    '4' : 'epoch_13',
    '5' : 'epoch_14'
}

In [13]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,2))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=2, bias=Tr

In [14]:
test_scores = []
for k,v in best_models_1fc.items():
    load_path = os.path.join(parent_directory,'6239512/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    loss , acc = test(model)
    test_scores.append(acc)

print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

[0.7315914489311164, 0.6817102137767221, 0.6318289786223278, 0.672209026128266, 0.6555819477434679]
mean =  0.6745843230403801
std =  0.033152202037753736


# testing 2 Fc working memory

In [15]:
train_scores = [0.690, 0.708, 0.673, 0.707, 0.706]
val_scores = [0.659,0.654, 0.662, 0.673, 0.657]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 0.6967999999999999
train std = 0.01361469794009398
val mean = 0.661
val std = 0.006542170893518456


In [16]:
best_models_2fc = {
    '1' : 'epoch_31',
    '2' : 'epoch_34',
    '3' : 'epoch_29',
    '4' : 'epoch_36',
    '5' : 'epoch_34'
}

In [17]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),
    nn.Dropout(),nn.Linear(256,2))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [18]:
test_scores = []
for k,v in best_models_2fc.items():
    load_path = os.path.join(parent_directory,'6297626/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    loss , acc = test(model)
    test_scores.append(acc)

print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

[0.6793349168646081, 0.6555819477434679, 0.6413301662707839, 0.6817102137767221, 0.6864608076009501]
mean =  0.6688836104513063
std =  0.017428918126217715


# Testing for 3 Fcs

In [19]:
train_scores = [0.696, 0.672, 0.673, 0.688, 0.690]
val_scores = [0.662,0.653, 0.662, 0.680, 0.650]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 0.6838
train std = 0.009599999999999956
val mean = 0.6614
val std = 0.010461357464497625


In [20]:
best_models_3fc = {
    '1' : 'epoch_74',
    '2' : 'epoch_50',
    '3' : 'epoch_73',
    '4' : 'epoch_67',
    '5' : 'epoch_34'
}

In [21]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),
    nn.Dropout(),nn.Linear(256,128),nn.ReLU(),nn.Dropout(),
    nn.Linear(128,2))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [22]:
test_scores = []
for k,v in best_models_3fc.items():
    load_path = os.path.join(parent_directory,'6297747/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    loss , acc = test(model)
    test_scores.append(acc)

print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

[0.6555819477434679, 0.6579572446555819, 0.6532066508313539, 0.6057007125890737, 0.6769596199524941]
mean =  0.6498812351543943
std =  0.023638679913776203


# Testing 4 FCs

In [23]:
train_scores = [0.645, 0.654, 0.645, 0.644, 0.635]
val_scores = [0.648,0.656, 0.646, 0.655, 0.629]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 0.6446
train std = 0.00601996677731697
val mean = 0.6468
val std = 0.009703607576566572


In [24]:
best_models_4fc = {
    '1' : 'epoch_55',
    '2' : 'epoch_72',
    '3' : 'epoch_70',
    '4' : 'epoch_61',
    '5' : 'epoch_49'
}

In [25]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),
    nn.Dropout(),nn.Linear(256,128),nn.ReLU(),nn.Dropout(),
    nn.Linear(128,64),nn.ReLU(), nn.Dropout(),nn.Linear(64,2))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [26]:
test_scores = []
for k,v in best_models_4fc.items():
    load_path = os.path.join(parent_directory,'1364551/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    loss , acc = test(model)
    test_scores.append(acc)

print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

[0.6104513064133017, 0.6365795724465558, 0.6318289786223278, 0.6104513064133017, 0.6033254156769596]
mean =  0.6185273159144893
std =  0.013148078387741603


# Testing 5 FCs

In [27]:
train_scores = [0.586, 0.623, 0.610, 0.500, 0.619]
val_scores = [0.624,0.647, 0.646, 0.533, 0.628]

print('train mean =',np.mean(train_scores))
print('train std =',np.std(train_scores))
print('val mean =',np.mean(val_scores))
print('val std =',np.std(val_scores))

train mean = 0.5875999999999999
train std = 0.04564471491859709
val mean = 0.6155999999999999
val std = 0.042325406081926725


In [28]:
best_models_5fc = {
    '1' : 'epoch_52',
    '2' : 'epoch_73',
    '3' : 'epoch_74',
    '4' : 'epoch_7',
    '5' : 'epoch_74'
}

In [29]:
model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,256),nn.ReLU(),nn.Dropout(0.25),
        nn.Linear(256,128),nn.ReLU(),nn.Dropout(0.25),nn.Linear(128,64),nn.ReLU()
        ,nn.Dropout(), nn.Linear(64,32),nn.ReLU(),nn.Dropout(),nn.Linear(32,2))
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): Network(
    (cv1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn3): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn4): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (cv5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn5): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=

In [30]:
test_scores = []
for k,v in best_models_5fc.items():
    load_path = os.path.join(parent_directory,'1369811/models_fold',k,v)
    model.load_state_dict(torch.load(load_path))
    loss , acc = test(model)
    test_scores.append(acc)

print(test_scores)
print('mean = ',np.mean(test_scores))
print('std = ',np.std(test_scores)) 

[0.6460807600950119, 0.6104513064133017, 0.6413301662707839, 0.47980997624703087, 0.6247030878859857]
mean =  0.6004750593824227
std =  0.0616370097496744
